In [11]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()

True

In [12]:
client = OpenAI()

In [13]:
def create_assistant(instruction):
    assistant = client.beta.assistants.create(
        name='Chatbot Hoteleiro',
        instructions=instruction,
        tools=[{'type':'file_search'}],
        model='gpt-3.5-turbo'
    )
    return assistant

In [22]:
def upload_file(path, file_type='binary'):
    read_type = 't' if file_type == 'text' else 'b'
    file = client.files.create(file=open(path, 'r' + read_type), purpose='assistants')
    return file

In [23]:
base_prompt_instruction = """Você é um atendente experiente de um hotel. \
    Responda perguntas variadas sobre hotéis, acomodações, pontos turíscos e outras \
    informações relevantes aos possíveis hóspedes.    
"""

In [25]:
PATH_HTML = 'pousada-porto-de-galinhas-booking.html'

upload_file(PATH_HTML)

FileObject(id='file-p0e4nVNJtMKfJaPZ6iIKqDsy', bytes=3022974, created_at=1721149230, filename='pousada-porto-de-galinhas-booking.html', object='file', purpose='assistants', status='processed', status_details=None)

In [26]:
assistant = create_assistant(base_prompt_instruction)

In [27]:
vector_store = client.beta.vector_stores.create(name='Paginas HTML de Hoteis')

In [28]:
file_paths = [PATH_HTML]
file_streams = [open(path, 'rb') for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)

completed


In [29]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={'file_search':{'vector_store_ids':[vector_store.id]}}
)

In [30]:
thread = client.beta.threads.create(
    messages=[
        {
            'role':'user',
            'content':'Que pousada é essa? O que a pousada tem de bom? Quais os quartos dela?'
        }
    ]
)

In [31]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

In [33]:
run.to_dict()

{'id': 'run_q2XYPXiVylehNcu7yTZOrnSO',
 'assistant_id': 'asst_UF0QP2rqXlRVbKzepcschYuD',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1721149623,
 'expires_at': None,
 'failed_at': 1721149630,
 'incomplete_details': None,
 'instructions': 'Você é um atendente experiente de um hotel.     Responda perguntas variadas sobre hotéis, acomodações, pontos turíscos e outras     informações relevantes aos possíveis hóspedes.    \n',
 'last_error': {'code': 'rate_limit_exceeded',
  'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'},
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1721149624,
 'status': 'failed',
 'thread_id': 'thread_vIfWbKOPMCtDJu0sxG